# Agent Streaming and Responses

Master agent streaming modes and structured outputs.

**What you'll learn:**
- Agent streaming: messages, updates, values modes
- Structured output: Type-safe responses with Pydantic

#### Agent Streaming Modes

| Mode | Use Case | Returns |
|------|----------|---------|
| **messages** | Real-time token display | Individual message chunks as generated |
| **updates** | Debugging agent flow | Node name + output after each node executes |
| **values** | Track full state | Complete state snapshot after each step |

In [ ]:
import sys
sys.path.append('../')

import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3
from scripts import base_tools

In [ ]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [ ]:
conn = sqlite3.connect("db/streaming_agent.db", check_same_thread=False)
checkpointer = SqliteSaver(conn)

agent = create_agent(
    model=model,
    tools=[base_tools.web_search],
    checkpointer=checkpointer
)

In [ ]:
# Stream mode: messages
for chunk in agent.stream(
    {'messages': [HumanMessage('Search for tech news')]},
    stream_mode='messages',
    config={'configurable': {'thread_id': 'stream_msg'}}
):
    print(chunk)

In [ ]:
# Stream mode: updates
for chunk in agent.stream(
    {'messages': [HumanMessage('Search for AI news')]},
    stream_mode='updates',
    config={'configurable': {'thread_id': 'stream_upd'}}
):
    print(chunk)

In [ ]:
# Stream mode: values
for step, chunk in enumerate(agent.stream(
    {'messages': [HumanMessage('What is ML?')]},
    stream_mode='values',
    config={'configurable': {'thread_id': 'stream_val'}}
)):
    print(f"Step {step}: {len(chunk.get('messages', []))} messages")

In [ ]:
from scripts.agent_utils import stream_agent_response

stream_agent_response(agent, 'Search for AI news', thread_id='stream_util')

## Structured Output

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional
from langchain.agents.structured_output import ToolStrategy, ProviderStrategy

class FinancialAnalysis(BaseModel):
    company: str = Field(description="Company name")
    stock_symbol: str = Field(description="Stock ticker")
    current_price: Optional[str] = Field(description="Current price", default=None)
    analysis: str = Field(description="Brief analysis")
    recommendation: str = Field(description="Buy/Hold/Sell")

structured_agent = create_agent(
    model=model,
    tools=[base_tools.web_search],
    response_format=ToolStrategy(FinancialAnalysis)  # ProviderStrategy
)

response = structured_agent.invoke({
    'messages': [HumanMessage('Analyze Tesla stock')]
})

response